## Runner Example

Initialize the runner by loading a config.


1. prepare the kafka and opensearch cluster

In [1]:
%%bash
# starting kafka container
docker compose -f ../../../../../examples/compose/docker-compose.yml down -v
docker compose -f ../../../../../examples/compose/docker-compose.yml up -d kafka opensearch dashboards

# creating the topic
docker exec -i kafka /bin/bash -c "/opt/bitnami/kafka/bin/kafka-topics.sh --create --topic consumer --bootstrap-server localhost:9092 --partitions 1 --replication-factor 1"

# waiting for kafka to be ready
sleep 30

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
 Container kafka  Stopping
 Container dashboards  Stopping
 Container dashboards  Stopped
 Container dashboards  Removing
 Container dashboards  Removed
 Container opensearch  Stopping
 Container opensearch  Stopped
 Container opensearch  Removing
 Container opensearch  Removed
 Container kafka  Stopped
 Container kafka  Removing
 Container kafka  Removed
 Network compose_kafka  Removing
 Network compose_opensearch  Removing
 Volume compose_opensearch-data  Removing
 Volume compose_opensearch-data  Removed
 Network compose_kafka  Removed
 Network compose_opensearch  Removed
 Network compose_kafka  Creating
 Network compose_kafka  Created
 Network compose_opensearch  Creating
 Network compose_opensearch  Created
 Volume "compose_opensearch-data"  Creating
 Volume "compose_opensearch-data"  Created
 Container kafka  Creating
 Container opensearch  Creating
 Container kafka  Created
 Container opensearch  Created
 Contai

Created topic consumer.


2. produce messages


In [2]:
%%bash
# producing 3 events to kafka topic consumer
docker exec -i kafka /bin/bash -c "echo '{\"message\": \"the message\"}' | /opt/bitnami/kafka/bin/kafka-console-producer.sh --bootstrap-server localhost:9092 --topic consumer "
# showing events in kafka
#docker exec -i kafka /bin/bash -c "/opt/bitnami/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic consumer --from-beginning --max-messages 10"


bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [ ]:
import sys
import logging
from logprep.ng.runner import Runner
from logprep.ng.util.configuration import Configuration

logging.basicConfig(
    level=logging.DEBUG,
    stream=sys.stdout
)

def get_config():
    return {
        "process_count": 2,
        "pipeline": [
            {
                "processor_0": {
                    "type": "ng_generic_adder",
                    "rules": [
                        {
                            "filter": "*",
                            "generic_adder": {"add": {"event.tags": "generic added tag"}},
                        }
                    ],
                }
            },
        ],
        "input": {
            "kafka": {
                "type": "ng_confluentkafka_input",
                "topic": "consumer",
                "kafka_config": {
                    "bootstrap.servers": "127.0.0.1:9092",
                    "group.id": "cgroup3",
                    "enable.auto.commit": "true",
                    "auto.commit.interval.ms": "10000",
                    "enable.auto.offset.store": "false",
                    "queued.min.messages": "100000",
                    "queued.max.messages.kbytes": "65536",
                    "statistics.interval.ms": "60000"
                }
            },
        },
        "output": {
            "my-opensearch": {
                "type": "ng_opensearch_output",
                "hosts": [
                    "127.0.0.1:9200"
                ],
                "default_index": "processed",
                "default_op_type": "create",
                "message_backlog_size": 1000,
                "timeout": 10000,
                "flush_timeout": 60,
                "user": "admin",
                "secret": "admin",
                "desired_cluster_status": ["green", "yellow"]
            },
        },
    }


runner = Runner(configuration=Configuration(**get_config()))
runner.run()

DEBUG:Component:Checking health of kafka
DEBUG:Processor:GenericAdder (processor_0) loaded 1 rules
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9200
DEBUG:urllib3.connectionpool:http://127.0.0.1:9200 "GET /_cluster/health HTTP/1.1" 200 0
INFO:opensearch:GET http://127.0.0.1:9200/_cluster/health [status:200 request:0.017s]
DEBUG:opensearch:> None
DEBUG:opensearch:< {"cluster_name":"opensearch-cluster","status":"green","timed_out":false,"number_of_nodes":1,"number_of_data_nodes":1,"discovered_master":true,"discovered_cluster_manager":true,"active_primary_shards":3,"active_shards":3,"relocating_shards":0,"initializing_shards":0,"unassigned_shards":0,"delayed_unassigned_shards":0,"number_of_pending_tasks":0,"number_of_in_flight_fetch":0,"task_max_waiting_in_queue_millis":0,"active_shards_percent_as_number":100.0}
DEBUG:Component:Checking health of my-opensearch
INFO:Sender:All outputs have been set up.
DEBUG:Component:Checking health of processor_0
INFO:Pipeline